In [5]:
import requests
import json
import time

# for pokemon in dict_response
all_url = "https://pokeapi.co/api/v2/pokemon-species/"
wanted_games = ["red", "blue", "leafgreen", "white"]
pokemon_by_version = {version: [] for version in wanted_games}

# limit the calls, use pagination
offset = 0
limit = 100
pokemon_count = 0

while True:
    try:
        dict_response = requests.get(f"{all_url}?offset={offset}&limit={limit}").json()
    except requests.exceptions.ConnectionError:
        time.sleep(5)
        continue

    get_pokemon = dict_response["results"]

    if len(get_pokemon) == 0:
        break

    for pokemon in get_pokemon:
        individual_pokemon = pokemon["url"]

        try:
            get_url = requests.get(individual_pokemon).json()
        except requests.exceptions.ConnectionError:
            time.sleep(5)
            continue

        if len(get_url["flavor_text_entries"]) > 0:
            for game_version in get_url["flavor_text_entries"]:
                version_name = game_version["version"]["name"]

                if version_name in wanted_games and game_version["flavor_text"]:
                    pokemon_by_version[version_name].append(pokemon["name"])

    if not dict_response["next"]:
        break
    else:
        offset += limit

# Save the results in a JSON file split on the version of the game
for version in wanted_games:
    filename = f"pokemon_by_{version}.json"
    data = {"pokemon": pokemon_by_version[version]}
    with open(filename, "w") as file:
        json.dump(data, file)


KeyboardInterrupt: 

In [13]:
import os
import requests
import json
import time

# for pokemon in dict_response
all_url = "https://pokeapi.co/api/v2/pokemon-species/"
wanted_games = ["red", "blue", "leafgreen", "white"]
pokemon_by_version = {version: [] for version in wanted_games}

# create directory if it doesn't exist
if not os.path.exists("pokemon_species"):
    os.makedirs("pokemon_species")

# limit the calls, use pagination
offset = 0
limit = 100
pokemon_count = 0

while True:
    try:
        dict_response = requests.get(f"{all_url}?offset={offset}&limit={limit}").json()
    except requests.exceptions.ConnectionError:
        time.sleep(5)
        continue

    get_pokemon = dict_response["results"]

    if len(get_pokemon) == 0:
        break

    for pokemon in get_pokemon:
        individual_pokemon = pokemon["url"]

        try:
            get_url = requests.get(individual_pokemon).json()

        except requests.exceptions.ConnectionError:
            time.sleep(5)
            continue

        if len(get_url["flavor_text_entries"]) > 0:
            for game_version in get_url["flavor_text_entries"]:
                version_name = game_version["version"]["name"]

                if version_name in wanted_games and game_version["flavor_text"]:
                    pokemon_by_version[version_name].append(pokemon["name"])

    if not dict_response["next"]:
        break
    else:
        offset += limit


In [14]:
# Save the result to a JSON file split on the version of the game
for version, pokemon_list in pokemon_by_version.items():
    data = {"version": version, "pokemon": pokemon_list}
    file_name = f"pokemon_species/{version}.json"
    with open(file_name, "w") as file:
        json.dump(data, file)

In [23]:
import os
import json

# define the folder containing the JSON files
folder_path = 'pokemon_species'

# create an empty dictionary to hold the sets
sets_dict = {}

# iterate over all files in the folder
for filename in os.listdir(folder_path):
    # check if the file is a JSON file
    if filename.endswith('.json'):
        # open the file and load its content as a dictionary
        with open(os.path.join(folder_path, filename), 'r') as f:
            file_content = json.load(f)
        
        # extract the relevant data from the dictionary and convert it to a set
        file_set = set(file_content['pokemon'])
        
        # add the set to the dictionary with the filename as the key
        sets_dict[filename] = file_set

pokemon_white = sets_dict["white.json"]
pokemon_blue = sets_dict["blue.json"]
pokemon_leaf_green = sets_dict["leafgreen.json"]
pokemon_red = sets_dict["red.json"]
print(len(pokemon_white))
print(len(pokemon_red))
print(len(pokemon_blue))
print(len(pokemon_leaf_green))

common_pokemon = pokemon_red | pokemon_blue | pokemon_leaf_green | pokemon_white
common_pokemon = pokemon_red & pokemon_blue & pokemon_leaf_green & pokemon_white

print(len(common_pokemon))
print(common_pokemon)




649
151
151
386
151
{'pidgeot', 'poliwhirl', 'tentacool', 'electrode', 'moltres', 'ponyta', 'drowzee', 'tentacruel', 'wigglytuff', 'articuno', 'slowpoke', 'ekans', 'dugtrio', 'hitmonchan', 'meowth', 'growlithe', 'muk', 'gengar', 'charmeleon', 'goldeen', 'pidgeotto', 'scyther', 'oddish', 'victreebel', 'charizard', 'onix', 'charmander', 'tangela', 'beedrill', 'vaporeon', 'haunter', 'zubat', 'rattata', 'machop', 'persian', 'exeggutor', 'gyarados', 'staryu', 'chansey', 'zapdos', 'magneton', 'rhydon', 'dewgong', 'poliwrath', 'jolteon', 'grimer', 'squirtle', 'spearow', 'omastar', 'caterpie', 'nidorina', 'vileplume', 'venonat', 'dratini', 'clefable', 'sandshrew', 'aerodactyl', 'horsea', 'bellsprout', 'golem', 'venomoth', 'gloom', 'venusaur', 'lickitung', 'psyduck', 'jigglypuff', 'ivysaur', 'marowak', 'weepinbell', 'starmie', 'snorlax', 'paras', 'poliwag', 'raticate', 'dragonair', 'mewtwo', 'krabby', 'bulbasaur', 'golduck', 'butterfree', 'dodrio', 'jynx', 'farfetchd', 'wartortle', 'magmar', 'h

In [ ]:
pokemon_url = "https://pokeapi.co/api/v2/pokemon"
pokemon_list = []

for pokemon in common_pokemon:
    while True:
        try:
            pokemon_data = requests.get(f"{pokemon_url}/{pokemon}").json()
            break
        except requests.exceptions.ConnectionError:
            time.sleep(5)
            continue

    wanted_slots = [1, 2]  # list
    available_slots = [type_dict["slot"] for type_dict in pokemon_data["types"]]
    # attack_types = [d["type"] for d in pokemon_data]

    # check =  all(item in available_slots  for item in wanted_slots)
    slots = []
    if any(slot in available_slots for slot in wanted_slots):
        for type_dict in pokemon_data["types"]:
            slot = type_dict.get("slot")
            attack_name = type_dict.get("type").get("name")
            slots.append({slot: attack_name})


    # dynamically populate slot 2

    stats = {
        "name": pokemon_data["name"].capitalize(),
        "id": pokemon_data["id"],
        "base_experience": pokemon_data["base_experience"],
        "weight": pokemon_data["weight"],
        "height": pokemon_data["height"],
        "order": pokemon_data["order"],
        "slot 1": slots[0].get(1),
        "bmi": round(pokemon_data["weight"] / (pokemon_data["height"] ** 2), 4),
        "front_default": pokemon_data["sprites"]["front_default"],
    }

    # remove slot 2 is empty
    # only add slot 2 if it exists
    # print(len(slots))

    if len(slots) > 1:
        stats["slot 2"] = slots[1].get(2)

        # Move "slot 2" after "slot 1" in the dictionary
        new_stats = {}
        for key, value in stats.items():
            if key == "slot 1":
                new_stats[key] = value
                if "slot 2" in stats:
                    new_stats["slot 2"] = stats["slot 2"]
            elif key != "slot 2":
                new_stats[key] = value

        stats = new_stats

    pokemon_list.append(stats)
    

print(len(pokemon_list))



In [1]:
pokemon_red = set(pokemon_by_version["red"])
pokemon_blue = set(pokemon_by_version["blue"])
pokemon_leaf_green = set(pokemon_by_version["leafgreen"])
pokemon_white = set(pokemon_by_version["white"])

print(len(pokemon_red))
print(len(pokemon_blue))
print(len(pokemon_leaf_green))
print(len(pokemon_white))

common_pokemon = pokemon_red | pokemon_blue | pokemon_leaf_green | pokemon_white


NameError: name 'pokemon_by_version' is not defined